In [1]:
# from ._conv import register_converters as _register_converters
import tensorflow as tf
import os
import time
import math
from datetime import datetime
slim = tf.contrib.slim
trunc_normal = lambda stddev : tf.truncated_normal_initializer(0.0, stddev)

/home/hhl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

* 定义经常用到的默认参数 如 卷积的集合函数、权重初始化方式、标准化器

In [3]:
def inception_v3_arg_scope(weight_decay=0.0004, stddev=0.1, batch_norm_var_collection='moving_vars'):
    batch_norm_params = {
        'decay' : 0.9997,
        'epsilon' : 0.0001,
        'updates_collections' : tf.GraphKeys.UPDATE_OPS,
        'variables_collections' : {
            'beta' : None,
            'gamma' : None,
            'moving_mean' : [batch_norm_var_collection],
            'moving_variance' : [batch_norm_var_collection],
        }
    }
    with slim.arg_scope([slim.conv2d, slim.fully_connected], weights_regularizer=slim.l2_regularizer(weight_decay)): 
        with slim.arg_scope(
            [slim.conv2d],
            weights_initializer=trunc_normal(stddev), 
            activation_fn=tf.nn.relu,
            normalizer_fn=slim.batch_norm, 
            normalizer_params=batch_norm_params) as sc: 
            return sc ## 注意这种代码缩进要搞对。不然很容易出错。

In [4]:
def inception_v3_base(inputs, scope=None):
    end_points = {}
    with tf.variable_scope(scope,'InceptionV3',[inputs]):
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], stride=1, padding='VALID'):
            # 正式定义Inception V3的网络结构。首先是前面的非Inception Module的卷积层
            # 使用了slim.arg_scope,使得slim.conv2d,slim.max_pool2d的一些初始化工作都已经做好，一行就可以定义一个卷积。
            # 输入：inputs , 299 x 299 x 3
            # 第一个参数为输入的tensor，第二个是输出的通道数，卷积核尺寸，步长stride，padding模式
            net = slim.conv2d(inputs, 32, [3, 3], stride=2, scope='Conv2d_1a_3x3') # 149 x 149 x 32
            net = slim.conv2d(net, 32, [3, 3], scope='Conv2d_2a_3x3') # 147 x 147 x 32
            net = slim.conv2d(net, 64, [3, 3], padding='SAME', scope='Conv2d_2b_3x3') # 147 x 147 x 64
            net = slim.max_pool2d(net, [3, 3], stride=2, scope='MaxPool_3a_3x3') # 73 x 73 x 64
            # 1x1 的卷积将相关性高、在同一空间位置但不同通道上但特征链接在一起。允许了通道间的信息组合且符合Hebbian原理
            net = slim.conv2d(net, 80, [1, 1], scope='Conv2d_3b_1x1')  # 73 x 73 x 80.
            net = slim.conv2d(net, 192, [3, 3], scope='Conv2d_4a_3x3') # 71 x 71 x 192.
            net = slim.max_pool2d(net, [3, 3], stride=2, scope='MaxPool_5a_3x3') # 35 x 35 x 192.
            
            # 上面部分代码一共有5个卷积层，2个池化层，实现了对图片数据的尺寸压缩，并对图片特征进行了抽象
            
        # Inception blocks
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], stride=1, padding='SAME'):
            
            # 下面是第一个Inception，包含了三个结构类似的Inception Module, Mixed_5b\Mixed_5c\Mixed_5d
            with tf.variable_scope('Mixed_5b'): # 第一个Inception Module名称。Inception Module有四个分支
                with tf.variable_scope('Branch_0'): # 第一个分支64通道的1*1卷积
                    branch_0 = slim.conv2d(net, 64, [1, 1], scope = 'Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'): # 第二个分支48通道1*1卷积，链接一个64通道的5*5卷积
                    branch_1 = slim.conv2d(net, 48, [1, 1], scope = 'Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
                with tf.variable_scope('Branch_2'): # 
                    branch_2 = slim.conv2d(net, 64, [1, 1], scope = 'Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope = 'Conv2d_0b_3x3')
                    branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope = 'Conv2d_0c_3x3')
                with tf.variable_scope('Branch_3'):# 第四个分支为3*3的平均池化，连接32通道的1*1卷积
                    branch_3 = slim.avg_pool2d(net, [3, 3], scope = 'AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 32, [1, 1], scope = 'Conv2_0b_1x1')
                # 第四个维度合并，即输出通道上合并。第一个为batch_size, 第二/三为图片尺寸
                # 通道合计： 64 + 64 + 96 + 32 = 256, same pading ,所以输出为 35x35x256
                # 如果正常从35x35x192变到35x35x256 需要参数量 35*35*192*256 = 60M
                # 但是现在参数量为：64 + 48 + 5*5*64 + 64 + 3*3*96*2 + 32 = 3.5k 参数量减少17k倍。
                # 同时计算量也减少很多。这就是Inception精妙之处。
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3) 
                
            with tf.variable_scope('Mixed_5c'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 64, [1, 1], scope='Con2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                    branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3, 3], scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1')
                # 与Mixed_5b 只有branch_3不一样，输出35x35x288
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
                
            with tf.variable_scope('Mixed_5d'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 64, [1, 1], scope='Con2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                    branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net,[3, 3], scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1')
                # 与Mixed_5b 只有branch_3不一样，输出35x35x288
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
            
             # 下面是第二个Inception，包含了5个的Inception Module,Mixed_6a\Mixed_6b\Mixed_6c\Mixed_6d\Mixed_6e
            with tf.variable_scope('Mixed_6a'):# stride=2, 图片被压缩。
                with tf.variable_scope('Branch_0'): 
                    branch_0 = slim.conv2d(net, 384, [3, 3], stride=2, padding='VALID', scope='Conv2d_0a_1x1') 
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1, 96, [3, 3], scope='Conv2d_0b_3x3')
                    branch_1 = slim.conv2d(branch_1, 96, [3, 3], stride=2, padding='VALID', scope='Conv2d_0c_3x3')
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID', scope='MaxPool_0a_3x3')
                # （35-3）/2 + 1 = 17, 最终 17*17*(384+96+288) = 17*17*768
                net = tf.concat([branch_0, branch_1, branch_2], 3)
            
            with tf.variable_scope('Mixed_6b'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 128, [1, 1], scope='Conv2d_01_1x1')
                    branch_1 = slim.conv2d(branch_1, 128, [1, 7], scope='Conv2d_0b_1x7')
                    branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
                with tf.variable_scope('Branch_2'):
                    # 串联 1x7 和 7x1 相当于 一个7x7，但是参数量减少为原来的（7+7）/7*7 = 2/7，同时多来一个非线性变换。
                    branch_2 = slim.conv2d(net, 128, [1,1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 128, [7, 1], scope='Conv2d_0b_7x1')
                    branch_2 = slim.conv2d(branch_2, 128, [1, 7], scope='Conv2d_0c_1x7')
                    branch_2 = slim.conv2d(branch_2, 128, [7, 1], scope='Conv2d_0d_7x1')
                    branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
                
            # 与Mixed_6b一样，只是将128 变为160 
            with tf.variable_scope('Mixed_6c'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_01_1x1')
                    branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_0b_1x7')
                    branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
                with tf.variable_scope('Branch_2'):
                    # 串联 1x7 和 7x1 相当于 一个7x7，但是参数量减少为原来的（7+7）/7*7 = 2/7，同时多来一个非线性变换。
                    branch_2 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0b_7x1')
                    branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_0c_1x7')
                    branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0d_7x1')
                    branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
            
            # 与Mixed_6c一样
            with tf.variable_scope('Mixed_6d'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_01_1x1')
                    branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_0b_1x7')
                    branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
                with tf.variable_scope('Branch_2'):
                    # 串联 1x7 和 7x1 相当于 一个7x7，但是参数量减少为原来的（7+7）/7*7 = 2/7，同时多来一个非线性变换。
                    branch_2 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0b_7x1')
                    branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_0c_1x7')
                    branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0d_7x1')
                    branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
            
               # 与Mixed_6c，Mixed_6d一样，最终输出17 x 17 x 768
            with tf.variable_scope('Mixed_6e'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_01_1x1')
                    branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_0b_1x7')
                    branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
                with tf.variable_scope('Branch_2'):
                    # 串联 1x7 和 7x1 相当于 一个7x7，但是参数量减少为原来的（7+7）/7*7 = 2/7，同时多来一个非线性变换。
                    branch_2 = slim.conv2d(net, 160, [1,1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0b_7x1')
                    branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_0c_1x7')
                    branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0d_7x1')
                    branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
            # 将Mixed_6e存储，作为 Auxiliary Cliassfier 的辅助分类.。 
            end_points['Mixed_6e'] = net
            
            with tf.variable_scope('Mixed_7a'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 192, [1, 1], scope = 'Conv2d_0a_1x1')
                    branch_0 = slim.conv2d(branch_0,320, [3, 3], stride=2, padding='VALID', scope='Conv2d_0b_3x3')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 192, [1, 1], scope = 'Conv2d_0a_1x1')
                    branch_1 = slim.conv2d(branch_1, 192, [1, 7], scope='Conv2d_0b_1x7')
                    branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
                    branch_1 = slim.conv2d(branch_1, 192, [3, 3], stride=2, padding='VALID', scope='Conv2d_0d_3x3')
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID', scope='MaxPool_0a_3x3')
                # 8x8x1280
                net = tf.concat([branch_0, branch_1, branch_2], 3)
                
            with tf.variable_scope('Mixed_7b'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 382, [1, 1], scope='Conv2d_0a_1x1')
                    branch_1_a = slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_1b_1x3')
                    branch_1_b = slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_2b_3x1')
                    branch_1 = tf.concat([branch_1_a, branch_1_b], 3)
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net, 448, [1, 1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2,384, [3, 3], scope='Conv2d_0b_3x3')
                    branch_2_a = slim.conv2d(branch_2, 384, [1, 3], scope='Conv2d_1c_1x3')
                    branch_2_b = slim.conv2d(branch_2, 384, [3, 1], scope='Conv2d_2c_3x1')
                    branch_2 = tf.concat([branch_2_a, branch_2_b], 3)
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net, [3, 3],scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
                # 8x8x2048
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
            
            with tf.variable_scope('Mixed_7c'):
                with tf.variable_scope('Branch_0'):
                    branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_0a_1x1')
                with tf.variable_scope('Branch_1'):
                    branch_1 = slim.conv2d(net, 382, [1, 1], scope='Conv2d_0a_1x1')
                    branch_1_a = slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_1b_1x3')
                    branch_1_b = slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_2b_3x1')
                    branch_1 = tf.concat([branch_1_a, branch_1_b], 3)
                with tf.variable_scope('Branch_2'):
                    branch_2 = slim.conv2d(net, 448, [1, 1], scope='Conv2d_0a_1x1')
                    branch_2 = slim.conv2d(branch_2,384, [3, 3], scope='Conv2d_0b_3x3')
                    branch_2_a = slim.conv2d(branch_2, 384, [1, 3], scope='Conv2d_1c_1x3')
                    branch_2_b = slim.conv2d(branch_2, 384, [3, 1], scope='Conv2d_2c_3x1')
                    branch_2 = tf.concat([branch_2_a, branch_2_b], 3)
                with tf.variable_scope('Branch_3'):
                    branch_3 = slim.avg_pool2d(net, [3, 3],scope='AvgPool_0a_3x3')
                    branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
                # 8x8x2048
                net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
    return net,end_points

In [5]:
def inception_v3(inputs, num_classes=1000, is_training=True, dropout_keep_prob=0.8, 
                 prediction_fn=slim.softmax, spatial_squeeze=True, reuse=None, scope='InceptionV3'):
    with tf.variable_scope(scope, 'InceptionV3', [inputs, num_classes],reuse=reuse) as scope:
        with slim.arg_scope([slim.batch_norm, slim.dropout], is_training=is_training):
            print(inputs.shape)
            net, end_points = inception_v3_base(inputs, scope=scope)
            with slim.arg_scope([slim.conv2d,slim.max_pool2d, slim.avg_pool2d], stride=1, padding='SAME'):
                aux_logits = end_points['Mixed_6e'] # 17x17x768
                with tf.variable_scope('AuxLogits'):
                    aux_logits = slim.avg_pool2d(aux_logits,[5, 5], stride=3, padding='VALID', scope='AvgPool_1a_5x5')
                    # 5x5x768
                    aux_logits = slim.conv2d(aux_logits, 128,[1, 1], scope='Conv2d_1b_1x1') # 5x5x768
                    aux_logits = slim.conv2d(aux_logits, 768, [5, 5], weights_initializer=trunc_normal(0.01),
                                            padding='VALID', scope='Conv2d_2a_5x5')
                    aux_logits = slim.conv2d(aux_logits, num_classes, [1, 1], activation_fn=None,
                                            normalizer_fn=None,weights_initializer=trunc_normal(0.001),
                                            scope='Conv2d_2b_1x1')
                    if spatial_squeeze:
#                         aux_logits = tf.square(aux_logits, [1, 2], name='SpatialSqueeze')
                        aux_logits = tf.squeeze(aux_logits, [1, 2], name='SpatialSqueeze')
                    end_points['AuxLogits'] = aux_logits
        with tf.variable_scope('Logits'):
            net = slim.avg_pool2d(net, [8, 8], padding='VALID',
                                  scope='AvgPool_1a_8x8')
            # 1 x 1 x 2048
            net = slim.dropout(net, keep_prob=dropout_keep_prob, scope='Dropout_1b')
            end_points['PreLogits'] = net
            # 2048
            logits = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, # 输出通道数1000
                                 normalizer_fn=None, scope='Conv2d_1c_1x1') # 激活函数和规范化函数设为空
            if spatial_squeeze: # tf.squeeze去除输出tensor中维度为1的节点
              logits = tf.squeeze(logits, [1, 2], name='SpatialSqueeze')
            # 1000
        end_points['Logits'] = logits
        end_points['Predictions'] = prediction_fn(logits, scope='Predictions') # Softmax对结果进行分类预测
    return logits, end_points

In [6]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time =time.time()
        _ = session.run(target)
        duration = time.time() - start_time;
        if i >= num_steps_burn_in:
            if not i % 10 :
                print('%s: step%d, duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration;
            total_duration_squared += duration*duration
        mn = total_duration / num_batches
        vr = total_duration_squared / num_batches - mn* mn
        sd = math.sqrt(vr)
        print('$%s : %s across %d steps, %.3f +/- %.3f sec /batch' % (datetime.now(), info_string, num_batches, mn, sd))

In [7]:
batch_size = 32
height, width = 299,299
inputs = tf.random_uniform((batch_size,height, width, 3))
with slim.arg_scope(inception_v3_arg_scope()):
    logits,end_points = inception_v3(inputs, is_training=True)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
num_batches = 100
time_tensorflow_run(sess,logits, 'Forward')

(32, 299, 299, 3)
$2018-11-11 14:49:23.619205 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:23.692213 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:23.766262 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:23.840284 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:23.913778 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:23.987508 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:24.061269 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:24.134657 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:24.208033 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
$2018-11-11 14:49:24.281725 : Forward across 100 steps, 0.000 +/- 0.000 sec /batch
2018-11-11 14:49:24.355039: step0, duration = 0.073
$2018-11-11 14:49:24.355148 : Forward across 100 steps, 0.001 +/- 0.007 sec /batch
$2018-11-11 14:49

$2018-11-11 14:49:30.471662 : Forward across 100 steps, 0.062 +/- 0.027 sec /batch
$2018-11-11 14:49:30.545396 : Forward across 100 steps, 0.063 +/- 0.026 sec /batch
$2018-11-11 14:49:30.619198 : Forward across 100 steps, 0.063 +/- 0.026 sec /batch
$2018-11-11 14:49:30.692427 : Forward across 100 steps, 0.064 +/- 0.025 sec /batch
$2018-11-11 14:49:30.766545 : Forward across 100 steps, 0.065 +/- 0.024 sec /batch
$2018-11-11 14:49:30.840636 : Forward across 100 steps, 0.065 +/- 0.023 sec /batch
$2018-11-11 14:49:30.913694 : Forward across 100 steps, 0.066 +/- 0.022 sec /batch
2018-11-11 14:49:30.988071: step90, duration = 0.074
$2018-11-11 14:49:30.988226 : Forward across 100 steps, 0.067 +/- 0.021 sec /batch
$2018-11-11 14:49:31.061697 : Forward across 100 steps, 0.068 +/- 0.020 sec /batch
$2018-11-11 14:49:31.135573 : Forward across 100 steps, 0.068 +/- 0.019 sec /batch
$2018-11-11 14:49:31.209282 : Forward across 100 steps, 0.069 +/- 0.017 sec /batch
$2018-11-11 14:49:31.282823 : Forw